In [1]:
import cv2 as cv
import pyrealsense2 as rs2
import open3d as o3d
import numpy as np
import time

from spygrt.stream import Recording, NoMoreFrames, DualRecording
from spygrt.tracker import Tracker
#rec1 = Recording('C:/Users/yben4607/clean/ResearchData/VISION/Youssef_Test/Recording_Session/121622061907_2022May24015518.bag')
#rec2 = Recording('C:/Users/yben4607/clean/ResearchData/VISION/Youssef_Test/Recording_Session/123122061443_2022May24015518.bag')
#rec1.load_calibration('C:/Users/yben4607/clean/ResearchData/VISION/Youssef_Test/RS_Calibration/121622061907_cal.txt')
#rec2.load_calibration('C:/Users/yben4607/clean/ResearchData/VISION/Youssef_Test/RS_Calibration/123122061443_cal.txt')
rec1 = Recording('E:/VISION/129/Recording_Session/121622061907_2023Feb13022044.bag')
rec2 = Recording('E:/VISION/129/Recording_Session/123122061443_2023Feb13022044.bag')
rec1.load_calibration('121622061907_cal.txt')
rec2.load_calibration('123122061443_cal.txt')




array([[-0.76596981, -0.47690493,  0.46312463, -0.43199882],
       [-0.65983385,  0.45945311, -0.61818653,  0.47658443],
       [ 0.0808824 , -0.76817775, -0.65726215,  0.49703872],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [2]:
d = DualRecording(rec1,rec2)

In [3]:
d.stream1._playback.current_status()

<playback_status.stopped: 3>

In [4]:
d.start_stream()

In [5]:
d.get_frames()

((<pyrealsense2.frame Z16 #1160 @1676258444174.644043>,
  <pyrealsense2.frame RGB8 #1194 @1676258444174.980957>),
 (<pyrealsense2.frame Z16 #1121 @1676258444195.972412>,
  <pyrealsense2.frame RGB8 #1123 @1676258444196.309326>))

In [8]:
t = Tracker(ref_surface=d.pcd)

[Open3D INFO] Picked point #1161557 (-0.026, 0.034, 0.16) to add in queue.
[Open3D INFO] Picked point #228252 (0.13, -0.086, 0.14) to add in queue.


In [2]:

t = Tracker(ref_surface=d.pcd)


[Open3D INFO] Picked point #195005 (0.089, -0.065, 0.16) to add in queue.
[Open3D INFO] Picked point #189889 (0.087, -0.062, 0.17) to add in queue.
[Open3D INFO] Remove picked point #189889 from pick queue.
[Open3D INFO] Remove picked point #195005 from pick queue.
[Open3D INFO] Picked point #209075 (0.095, -0.071, 0.15) to add in queue.
[Open3D INFO] Picked point #1162800 (-0.031, 0.056, 0.16) to add in queue.


In [3]:
o3d.visualization.draw_geometries([t.ref_surface.to_legacy()])

In [2]:
from datetime import timedelta
delta = timedelta(seconds=70)
d.stream1.seek(delta)
d.stream2.seek(delta)

In [7]:
p1 = [0.3,0.3,0.3]
p2 = [-0.3,-0.3,0.2]
midz = np.mean([p1[2], p2[2]])
lower = [np.min([p1[0], p2[0]]), np.min([p1[1], p2[1]]), midz - 0.02]

In [8]:
 lower

[-0.3, -0.3, 0.23]

In [10]:
type(d.pcd.point['positions'].numpy()[0][0])

numpy.float32

In [9]:
t.get_motion(d.compute_pcd(True))

[[0.999999 -0.00117451 -0.000221870 -7.26435e-05],
 [0.00117435 0.999999 -0.000746738 0.000187356],
 [0.000222747 0.000746477 1.0 -8.60738e-05],
 [0.0 0.0 0.0 1.0]]
Tensor[shape={4, 4}, stride={4, 1}, Float64, CUDA:0, 0x130a509c00]

In [7]:
a=d.compute_pcd(True)

In [10]:
a is None

True

In [14]:
d.get_frames()

((<pyrealsense2.frame Z16 #8056 @1653364518745.431152>,
  <pyrealsense2.frame RGB8 #8052 @1653364518728.857178>),
 (<pyrealsense2.frame Z16 #8013 @1653364518748.400391>,
  <pyrealsense2.frame RGB8 #8008 @1653364518732.104736>))

In [15]:
d.pose is None

False

In [18]:
d.stream1.compute_pcd() + d.stream2.compute_pcd()

PointCloud on CPU:0 [1843200 points (Float32)].
Attributes: colors (dtype = UInt8, shape = {1843200, 3}).

In [4]:
t.ref_surface.estimate_normals(30,0.001)

In [8]:
Tracker.compute_rotation(t._t_guess)

AttributeError: type object 'Tracker' has no attribute 'isR_matrix'

In [ ]:
#data = []
import time
a = time.time()
mat_arr = []
for i in range(120000):
    
    try:
        t.get_motion(d.compute_pcd(True))
    except RuntimeError:
        continue
    except NoMoreFrames:
        np.savetxt('track.txt',data)
        break
    data.append(np.concatenate(([d.timestamp], Tracker.compute_translation(t.t_guess), Tracker.compute_rotation(t.t_guess))))
    mat_arr.append(t._t_guess)
    if i%100 == 0:
        print(i/100)
        np.savetxt('track.txt',data)
b = time.time()
b-a   

0.0
1.0
Timestamps:1676258453347.0708 1433 1676258453469.0671 1397
Timestamps:1676258454422.1592 1465 1676258454544.2842 1429
Timestamps:1676258454422.1592 1465 1676258454544.2842 1429
Timestamps:1676258455194.96 1488 1676258455317.0886 1452
Timestamps:1676258457782.2197 1565 1676258457904.2498 1529
2.0
Timestamps:1676258458823.7617 1596 1676258458945.8157 1560
Timestamps:1676258461713.4575 1682 1676258461835.3643 1646
3.0
Timestamps:1676258467996.565 1869 1676258468118.3394 1833
4.0
Timestamps:1676258473540.3994 2034 1676258473662.154 1998
Timestamps:1676258475455.641 2091 1676258475577.3076 2055
Timestamps:1676258475455.641 2091 1676258475577.3076 2055
5.0
Timestamps:1676258479756.5095 2219 1676258479642.7573 2176
Timestamps:1676258479857.305 2222 1676258479743.5542 2179
Timestamps:1676258480327.6821 2236 1676258480213.9392 2193
Timestamps:1676258480327.6821 2236 1676258480213.9392 2193
Timestamps:1676258480663.6707 2246 1676258480785.121 2210
Timestamps:1676258483284.4224 2324 16762

In [4]:
o3d.visualization.draw_geometries([t._source.to_legacy()])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [8]:
t.t_guess

[[0.960616 -0.26193 -0.0927886 0.0517465],
 [0.276662 0.932738 0.23121 -0.0737735],
 [0.0259866 -0.247775 0.968469 0.0439792],
 [0.0 0.0 0.0 1.0]]
Tensor[shape={4, 4}, stride={4, 1}, Float64, CPU:0, 0x24d64541970]

In [7]:
saved_data = data

In [17]:
for i in range(48):
    d.compute_pcd(True)

In [18]:
t.get_motion(d.pcd)

[[0.976428 -0.135231 0.168226 -0.0104739],
 [0.146373 0.987664 -0.0556359 -0.0120752],
 [-0.158627 0.0789482 0.984177 -0.068689],
 [0.0 0.0 0.0 1.0]]
Tensor[shape={4, 4}, stride={4, 1}, Float64, CPU:0, 0x18e6fd1f110]

In [19]:
o3d.visualization.draw_geometries([t._source.to_legacy()])

In [3]:
np.savetxt('data.txt',np.asarray(data))

In [5]:
d.timestamp

1676258858178.8535

In [4]:
len(data)

496

In [7]:
a=1.676258596709123047e+12
b=1.676258444235704590e+12
a-b

152473.41845703125

In [5]:
import numpy as np
R = np.asarray([[1,2,3,4],[5,6,7,8],[9, 10 ,11, 12],[13,14,15,16]])

In [6]:
R

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12],
       [13, 14, 15, 16]])

In [7]:
R[0:3].T[0:3].T

array([[ 1,  2,  3],
       [ 5,  6,  7],
       [ 9, 10, 11]])

In [9]:
(1676258858178.8535-a)/1000/60

4.3578288411458335

In [10]:
np.savetxt('track.txt',data)

In [10]:
a = o3d.core.Tensor(np.asarray([]),device=o3d.core.Device('cuda:0'))

In [11]:
a.is_cuda

True

In [7]:
1.676259139169756836e+12-1.676258444218906494e+12

694950.8503417969

In [6]:
I = np.asarray([[1,0,0,0.022],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
rec1._pose = I@rec1._pose
rec2._pose = I@rec2._pose

In [7]:
d.compute_pcd(False)

PointCloud on CUDA:0 [1843200 points (Float32)].
Attributes: colors (dtype = UInt8, shape = {1843200, 3}).

In [11]:
data = []
data.append(np.concatenate(([d.timestamp], Tracker.compute_translation(t.t_guess), Tracker.compute_rotation(t.t_guess))))


In [12]:
data

[array([ 1.67625844e+12, -7.26434555e-05,  1.87355803e-04, -8.60738479e-05,
         6.72852432e-02, -1.27624544e-02,  4.27700061e-02])]